In [45]:

import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from qiskit_aer import Aer
from qiskit.circuit.library import PhaseOracle
from qiskit_aer.primitives import Sampler
from qiskit_algorithms.amplitude_amplifiers.grover import Grover
from qiskit_algorithms.amplitude_amplifiers.amplitude_amplifier import AmplificationProblem


ImportError: cannot import name 'BaseSampler' from 'qiskit.primitives' (c:\Users\tikes\miniconda3\Lib\site-packages\qiskit\primitives\__init__.py)

In [34]:
# ----- Quantum Minimum Finding -----

def index_to_bin(index, num_bits):
    return format(index, f'0{num_bits}b')

def create_oracle_expression(min_index, num_bits):
    bin_index = index_to_bin(min_index, num_bits)
    expr = ' & '.join([f"{'' if bit == '1' else '~'}x{i}" for i, bit in enumerate(bin_index)])
    return expr

def grover_find_min_index(values):
    n = len(values)
    num_bits = int(np.ceil(np.log2(n)))
    padded_length = 2 ** num_bits

    padded_values = values + [float('inf')] * (padded_length - n)
    min_index = np.argmin(padded_values)

    oracle_expr = create_oracle_expression(min_index, num_bits)
    oracle = PhaseOracle(oracle_expr)
    problem = AmplificationProblem(oracle)

    backend = Aer.get_backend("aer_simulator")
    sampler = Sampler()
    grover = Grover(sampler=sampler)
    result = grover.amplify(problem)

    measured_index = max(result.measurements.items(), key=lambda x: x[1])[0]
    return int(measured_index, 2)

def quantum_sort_cluster(cluster_df, sort_column):
    df = cluster_df.copy().reset_index(drop=True)
    sorted_rows = []

    while not df.empty:
        values = df[sort_column].tolist()
        min_idx = grover_find_min_index(values)
        sorted_rows.append(df.loc[min_idx])
        df = df.drop(min_idx).reset_index(drop=True)

    return pd.DataFrame(sorted_rows)

# ----- Main Cluster-Based Hybrid Sort -----

def cluster_based_quantum_sort(input_csv, sort_column, n_clusters=20, output_csv='cluster_sorted.csv'):
    df = pd.read_csv(input_csv)
    df = df.dropna()
    
    if sort_column not in df.columns:
        print(f"Column '{sort_column}' not found.")
        return

    print("Original Data:\n", df)

    # Clustering
    clustering_data = df[[sort_column]]
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    df['cluster'] = kmeans.fit_predict(clustering_data)

    all_sorted = []
  
    for cluster_id in range(n_clusters):
        cluster_df = df[df['cluster'] == cluster_id].drop(columns=['cluster'])
        print(f"\nSorting Cluster {cluster_id} (size {len(cluster_df)}):")
        sorted_cluster = quantum_sort_cluster(cluster_df, sort_column)
        all_sorted.append(sorted_cluster)

    # Combine clusters and final classical sort
    merged_df = pd.concat(all_sorted, ignore_index=True)
    final_sorted_df = merged_df.sort_values(by=sort_column).reset_index(drop=True)

    print("\nFinal Sorted Data:")
    print(final_sorted_df)

    final_sorted_df.to_csv(output_csv, index=False)
    print(f"\nSorted data saved to '{output_csv}'.")

# Example usage
if __name__ == "__main__":
    cluster_based_quantum_sort('phpB0xrNj.csv', sort_column="f2", n_clusters=2)

Original Data:
           f1      f2      f3      f4      f5      f6      f7      f8      f9  \
0    -0.4394 -0.0930  0.1718  0.4620  0.6226  0.4704  0.3578  0.0478 -0.1184   
1    -0.4348 -0.1198  0.2474  0.4036  0.5026  0.6328  0.4948  0.0338 -0.0520   
2    -0.2330  0.2124  0.5014  0.5222 -0.3422 -0.5840 -0.7168 -0.6342 -0.8614   
3    -0.3808 -0.0096  0.2602  0.2554 -0.4290 -0.6746 -0.6868 -0.6650 -0.8410   
4    -0.3412  0.0946  0.6082  0.6216 -0.1622 -0.3784 -0.4324 -0.4358 -0.4966   
...      ...     ...     ...     ...     ...     ...     ...     ...     ...   
7331 -0.6842 -0.3280 -0.1984  0.2956  0.8786  0.8948  0.3118  0.1822  0.1012   
7332 -0.5912 -0.2420  0.8174  1.0000  0.4642  0.6428  0.6944  0.3056 -0.3888   
7333 -0.6696 -0.3730  0.1584  0.8910  1.0000  0.9762  0.9762  0.7684  0.4106   
7334 -0.5764 -0.1764  0.5106  0.3742 -0.1670 -0.5858 -0.7882 -0.7224 -0.6330   
7335 -0.6624 -0.3334  0.3666  0.4292 -0.2084 -0.5374 -0.4542 -0.6208 -0.6376   

         f10  ...    f6

NameError: name 'AmplificationProblem' is not defined